# Distortion Correction

In [1]:
import numpy as np
import cv2
import glob
import pickle
import matplotlib.pyplot as plt

import matplotlib.image as mpimg
%matplotlib inline

#Load calibrated camera
Cam_Calib=pickle.load( open("CamCal/camCal_pickle.p", "rb"))

save_dir= r'output_images/pipeline/'

cam_dist=Cam_Calib["dist"]
cam_mtx=Cam_Calib["mtx"]

def undistort_s(img, dist=cam_dist, mtx=cam_mtx):
    '''Undistorts color image, ouputs undistorted img, saturation channel'''
    dst=cv2.undistort(img, cam_mtx, cam_dist, None, cam_mtx)
    HLS_img=cv2.cvtColor(dst, cv2.COLOR_RGB2HLS)
    S_channel= HLS_img[:,:,2]
    return dst, S_channel


# Gradient Threshold

In [2]:
#Diagonal sobel kernels from http://homepages.inf.ed.ac.uk/rbf/HIPR2/linedet.htm
up_kernel = [[-1.,-1.,2.],
         [-1.,2.,-1.],
         [2.,-1.,-1.]]
up_kernel=np.asarray(up_kernel)/12.0

down_kernel = [[2.,-1.,-1.],
         [-1.,2.,-1.],
         [-1.,-1.,2.]]
down_kernel=np.asarray(down_kernel)/12.0

def abs_sobel_thresh(gray, orient='x', thresh_min=0, thresh_max=255, sobel_kernel=3):
    """Axis sobel, asumes img is gray image, returns binary image"""
    # Apply the following steps to img
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient=='x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    elif orient=='y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    elif orient=='+':
        sobel = cv2.filter2D(gray,-1,up_kernel)
    elif orient=='-':
        sobel = cv2.filter2D(gray,-1,down_kernel)
    else:
        raise "Not valid orientation!"
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel=np.uint8(255 * abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
    binary_output=np.zeros_like(scaled_sobel)
            # is > thresh_min and < thresh_max
    # 6) Return this mask as your binary_output image
    binary_output[(scaled_sobel>thresh_min) & (scaled_sobel<thresh_max)] = 1
    return binary_output

def hls_tresh(S_channel, low_limit=0, high_limit=255):
    """Takes saturation channel, outputs binary image"""
    s_binary=np.zeros_like(S_channel)
    s_binary[(S_channel>=low_limit) & (S_channel<=high_limit)]=1
    return s_binary

def multiple_threshold(image, S_channel, min_dt=0, max_dt=255, s_low=128, s_high=255, k_slope1=up_kernel, k_slope2=down_kernel):
    """Takes gray image and Saturation channel, outputs binary image"""
    # Apply the following steps to img
    # 1) Apply sobel in direction
#    #X not used, noisy
    bin_diag=abs_sobel_thresh(image, orient='x', thresh_min=32, thresh_max=255, sobel_kernel=5)
    #Less significant on Y axis , lots of noise
    #Diagonal Sobels!
#    pos=abs_sobel_thresh(image, orient='+', thresh_min=min_dt, thresh_max=max_dt)
#    neg=abs_sobel_thresh(image, orient='-', thresh_min=min_dt, thresh_max=max_dt)
#    bin_diag=np.zeros_like(pos)
#    bin_diag[(pos==1)|(neg==1)]=1
    #Close gaps in binary image
    bin_diag=cv2.morphologyEx(bin_diag, cv2.MORPH_CLOSE, np.ones((7,3),np.uint8))
#    
    #Even smaller magnitudes cause noise, not using magnitude sobel
    #Lots of noise (even when combining magnitude and orientation), not using orientation sobel
    # 4) Saturation channel
    s_bin= hls_tresh(S_channel ,low_limit=s_low, high_limit=s_high)
    #Open big spaces, as to desaturate the saturated channel (bad_joke_here)
    s_bin=cv2.morphologyEx(s_bin, cv2.MORPH_OPEN, np.ones((1,3),np.uint8))   
    # 5)Mix 2 above
    binary_output=np.zeros_like(bin_diag)
    binary_output[((bin_diag==1)|(s_bin==1))]=1
    #binary_output=bin_diag
    binary_output= cv2.erode(binary_output,np.ones((1,3),np.uint8),iterations = 1)
    return binary_output

# Prespective transform

In [3]:
#Polygon borders (Original IMG)
src_pts = np.array([[230,700],[610,440],[670,440],[1080,700]], np.float32)
#Destination Points
dst_pts = np.array([[300,720],[300,-200],[950,-200],[950,720]], np.float32)

def birds_eye(img, src=src_pts, dst=dst_pts):
    """Transform image to birdseye"""
    M = cv2.getPerspectiveTransform(src, dst)
    bird = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_NEAREST)
    return bird

def region_of_interest(img):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """    
    shape = img.shape
    vertices = np.array([[(0,0),(shape[1],0),(shape[1],0),(6*shape[1]/7,shape[0]),
                      (shape[1]/7,shape[0]), (0,0)]],dtype=np.int32)

    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


# Line detect

In [4]:
#150
window_width= 150
#window_height=20
window_height=72
#50
margin=80

def window_mask(width, height, img_ref, center, level):
    """Get window mask"""
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output
    
def find_window_centroids(warped, window_width, window_height, margin, custom_window=None):
    """Find centroids of windows in image, returns windows centroids and central point of lanes"""
    # Add here condition for custom distribution (not implemented)
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    #Custom window, triangular shape as to push the weight of the convolution to the center
    for i in range(int(len(window)/2)):
        window[i]=window[i]+i
        window[-i]=window[-i]+i
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    # Sum ocatve bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(warped.shape[0]*1/6):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(warped.shape[0]*1/6):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    #Center of both lane markings
    center=r_center-l_center
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer, mode='full')
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        max_conv = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        if conv_signal[int(max_conv)]>0:
            l_center=max_conv
        else:
            l_center=l_center
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        max_conv = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        if conv_signal[int(max_conv)]>0:
            r_center=max_conv
        else:
            r_center=r_center
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))
    return window_centroids, center

def draw_regression(warped, window_centroids, window_height):
    """Draws regression on image"""
    if len(window_centroids) > 0:
        
        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)
        #get pixels for regression (currently unused)
        nonzero=warped.nonzero()
        nonzerox=np.array(nonzero[0])
        nonzeroy=np.array(nonzero[1])
        left_inds=[]
        right_inds=[]
        # Go through each level and draw the windows 
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
            #Setting boundaries
            topy=720 - level*window_height
            botomy=720- (level+1)*window_height
            x_left_low=int(window_centroids[level][0]-window_width/2)
            x_left_high=int(window_centroids[level][0]+window_width/2)
            x_right_low=int(window_centroids[level][1]-window_width/2)
            x_right_high=int(window_centroids[level][1]+window_width/2)
            
            good_left_inds = ((nonzerox >= botomy) & (nonzerox < topy) & (nonzeroy >= x_left_low) & (nonzeroy < x_left_high)).nonzero()[0]
            good_right_inds = ((nonzerox >= botomy) & (nonzerox < topy) & (nonzeroy >= x_right_low) & (nonzeroy < x_right_high)).nonzero()[0]
            
            left_inds.append(good_left_inds)
            right_inds.append(good_right_inds)
            
        # Draw windows        
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channle 
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((255*warped,255*warped,255*warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results

        #Get polinomials
        left_inds=np.concatenate(left_inds)
        right_inds=np.concatenate(right_inds)
        
        leftx=nonzerox[left_inds]
        lefty=nonzeroy[left_inds]
        rightx=nonzerox[right_inds]
        righty=nonzeroy[right_inds]
        
        left_fit=np.polyfit(leftx, lefty, 2)
        right_fit=np.polyfit(rightx, righty, 2)
        
        #Draw polinomial lines
        ploty =np.linspace(0, warped.shape[0]-1, warped.shape[0])
        left_fitx=(left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2])
        right_fitx = (right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2])
        left_line=[]
        right_line=[]
        for y,l,r in zip(ploty, left_fitx, right_fitx):
            left_line.append([np.int32(l), np.int32(y)])
            right_line.append([np.int32(r), np.int32(y)])
        pts1=np.array(left_line, np.int32)
        pts2=np.array(right_line, np.int32)
        pts1.reshape((-1,1,2))
        pts2.reshape((-1,1,2))
        output=cv2.polylines(output, [pts1, pts2], False, (255,0,0), 3)
    # If no window centers found, just display orginal road image and empty polynom
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
        print("No centers Found!")
        right_fit=[]
        left_fit=[]
    return output, left_fit, right_fit

def tangent_circle(poly_l, poly_r, pic_center, eval_point=720):
    """Takes polynomial regressions and lanes center, returns curvature and distance to center [m]"""
    #pixels to meter ratio
    ym_per_pix = 3.0/106
    xm_per_pix = 3.7/650
    ploty = np.linspace(0, 719, num=720)
    leftx=np.array([poly_l[0]*(y**2)+poly_l[1]*(y)+poly_l[0] for y in ploty])
    rightx=np.array([poly_r[0]*(y**2)+poly_r[1]*(y)+poly_r[0] for y in ploty])
    
    center=(pic_center-700)*xm_per_pix

    poly_ml=np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    poly_mr=np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    eval_point*=ym_per_pix
    
    curv_l = ((1 + (2*poly_ml[0]*eval_point + poly_ml[1])**2)**1.5) / np.absolute(2*poly_ml[0])
    curv_r = ((1 + (2*poly_mr[0]*eval_point + poly_mr[1])**2)**1.5) / np.absolute(2*poly_mr[0])
    
    return curv_l, curv_r, center

def unwarp_color(warped_color, left_fit, right_fit, text, src=dst_pts, dst=src_pts):
    """Takes color image, returns identified poly, unwarps to original"""
    #Draw polinomial lines
    ploty =np.linspace(0, warped_color.shape[0]-1, warped_color.shape[0])
    left_fitx=(left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2])
    right_fitx = (right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2])
    left_line=[]
    right_line=[]
    for y,l,r in zip(ploty, left_fitx, right_fitx):
        left_line.append([np.int32(l), np.int32(y)])
        right_line.append([np.int32(r), np.int32(y)])
    pts1=np.array(left_line, np.int32)
    pts2=np.array(right_line, np.int32)
    pts1.reshape((-1,1,2))
    pts2.reshape((-1,1,2))
    total_pts=[]
    total_pts.append(pts1)
    total_pts.append(list(reversed(pts2)))
    total_pts=np.array(total_pts).reshape((-1,1,2))
    curvature_txt="Curvature: "+str(text[0])[:5] +"[m]"
    center_txt="Distance to lane Center: "+str(text[2])[0:5]+"[m]"
    output=np.zeros_like(warped_color)
    output=cv2.fillPoly(output, [total_pts], color=(63,79,232))
    output=cv2.polylines(output, [pts1, pts2], False, (13,26,129), 20)
#    output=cv2.fillPoly(output, [total_pts], color=(255,0,255))
#    output=cv2.polylines(output, [pts1, pts2], False, (255,0,0), 20)
    M = cv2.getPerspectiveTransform(src, dst)
    output = cv2.warpPerspective(output, M, (warped_color.shape[1], warped_color.shape[0]), flags=cv2.INTER_NEAREST)
    output = cv2.putText(output,curvature_txt, (50,50), cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255,255,255))
    output = cv2.putText(output,center_txt, (50,100), cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255,255,255))
    return output
    
    

In [10]:
#Full pipeline
buffer_left=[]
buffer_right=[]
last_data=[]
no_fault=0

def sanity(left_poly, right_poly):
    if len(last_data)<1:
        return True
    if right_poly[2]-left_poly[2]<200:
        return False
    left_sane=np.allclose(left_poly[2], last_data[0][2], atol=200)
    right_sane=np.allclose(right_poly[2], last_data[1][2], rtol=200)
    return left_sane & right_sane

def evaluate_buffer(poly_l, poly_r):
    global buffer_left
    global buffer_right
    buffer_left.append(poly_l)
    buffer_right.append(poly_r)
    if len(buffer_left)>5:
        buffer_left=buffer_left[1:]
        buffer_right=buffer_right[1:]
    
    mean_fit_l=np.mean(np.asarray(buffer_left), axis=0)
    mean_fit_r=np.mean(np.asarray(buffer_right), axis=0)
    #print("Old fit:" poly_l)
    #print("New fit:" mean_fit_l)
    return mean_fit_l, mean_fit_r


def new_pipeline(camera_image, window_width=window_width, window_height=window_height, margin=margin, save_step=False, save_dir=r'output_images/pipeline/', prefix='0', buffered=False):
    image, S_channel = image, S_channel=undistort_s(camera_image)
    warp_color= birds_eye(image)
    warp_S= birds_eye(S_channel)
    gray=cv2.cvtColor(warp_color, cv2.COLOR_RGB2GRAY)
    gray=cv2.blur(gray,(3,3))
    bin_im=multiple_threshold(gray, warp_S)
    bin_roi=region_of_interest(bin_im)
    window_centroids, pic_center = find_window_centroids(bin_roi, window_width, window_height, margin)
    detect_lines, left_fit, right_fit =draw_regression(bin_roi, window_centroids, window_height)
    if buffered:
        global no_fault
        global last_data
        if sanity(left_fit, right_fit):
            last_data=[left_fit, right_fit]
            no_fault=0
            left_fit, right_fit = evaluate_buffer(left_fit, right_fit)
        else:
            no_fault+=1
            print("Consecutive Fault: {}".format(no_fault))
            print("Last good data")
            print(last_data)
            print("wrong data")
            print(left_fit, right_fit)
            if no_fault<15:
                left_fit, right_fit= last_data
            else:
                print("Recalibrating")
                last_data=[left_fit, right_fit]
                no_fault=0
        
    curv_l, curv_r, center = tangent_circle(left_fit, right_fit, pic_center)
    text_array=[curv_l, curv_r, center]
    detect_color=unwarp_color(warp_color, left_fit, right_fit, text_array)
    result=cv2.addWeighted(image, 1, detect_color, .8, 0)
    

    if save_step:
        cv2.imwrite(save_dir+prefix+"undist.jpg", image)
        cv2.imwrite(save_dir+prefix+"S_channel.jpg", 255*S_channel)
        cv2.imwrite(save_dir+prefix+"warp_color.jpg", warp_color)
        cv2.imwrite(save_dir+prefix+"gray.jpg", gray)
        cv2.imwrite(save_dir+prefix+"warp_roi.jpg", 255*warp_S)
        cv2.imwrite(save_dir+prefix+"bin.jpg", 255*bin_im)
        cv2.imwrite(save_dir+prefix+"bin_roi.jpg", 255*bin_roi)
        cv2.imwrite(save_dir+prefix+"detected.jpg", detect_lines)
        cv2.imwrite(save_dir+prefix+"detected_color.jpg", detect_color)
        cv2.imwrite(save_dir+prefix+"Result.jpg", result)
    
    return result, left_fit, right_fit, text_array

def frame_only(camera_image):
    img, left_fit, right_fit, text_array = new_pipeline(camera_image, buffered=True)
    return img

In [6]:
img_files = glob.glob('test_images/*.jpg')
history=[]
for i, file in enumerate(img_files):
    camera_image=cv2.imread(file)
    history.append(new_pipeline(camera_image, save_step=True, prefix=str(i))[1:])
    

In [11]:
#for i in history:
#    print (i)

#Video
from moviepy.editor import VideoFileClip
#from IPython.display import HTML

video_output="Test.mp4"
clip1=VideoFileClip("project_video.mp4")

video_clip=clip1.fl_image(frame_only)
%time video_clip.write_videofile(video_output, audio=False)


[MoviePy] >>>> Building video Test.mp4
[MoviePy] Writing video Test.mp4


 45%|███████████████████████████████████▍                                           | 566/1261 [02:13<03:05,  3.75it/s]

Consecutive Fault: 1
Last good data
[array([ -2.99764390e-04,   3.25826408e-01,   2.26070388e+02]), array([ -5.66954547e-05,   6.39867518e-02,   9.74071364e+02])]
wrong data
[ -4.76890237e-04   8.30477230e-01  -1.32788605e+01] [ -4.75753013e-05   5.70256629e-02   9.64032472e+02]


 46%|████████████████████████████████████▏                                          | 577/1261 [02:15<02:08,  5.34it/s]

Consecutive Fault: 1
Last good data
[array([  2.12672608e-04,  -2.58598647e-01,   3.92397144e+02]), array([  1.22954499e-04,  -1.10101533e-01,   1.02427003e+03])]
wrong data
[ -8.35465547e-04   1.04901903e+00  -9.95702343e+00] [  1.15862860e-04  -1.04303922e-01   1.01622064e+03]


...

Consecutive Fault: 15
Last good data
[array([  1.36003918e-03,  -1.38690984e+00,   6.14656028e+02]), array([  1.32234746e-03,  -1.44599148e+00,   1.34911032e+03])]
wrong data
[  8.13120348e-04  -9.68612991e-01   5.90209493e+02] [  1.26566109e-03  -4.56407265e-01   4.90699254e+02]
Recalibrating


 83%|████████████████████████████████████████████████████████████████▉             | 1049/1261 [03:52<00:57,  3.66it/s]

Consecutive Fault: 1
Last good data
[array([  6.75811496e-04,  -8.39944244e-01,   5.57167525e+02]), array([  4.63067820e-04,  -7.95014989e-01,   1.32224836e+03])]
wrong data
[  5.25759679e-04  -6.86642654e-01   5.12464197e+02] [ -2.13727789e-03   2.95593207e+00  -1.25097718e+02]


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [04:38<00:00,  5.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Test.mp4 

Wall time: 4min 40s


In [8]:
for j in history:
    print(j[0])

print(len(history))
acumulator_l=[]
for i in range(len(history)):
    acumulator_l.append(history[i][0])
    if len(acumulator_l)>3:
        acumulator_l=acumulator_l[1:]
    print("Acumulator has {} elements".format(len(acumulator_l)))
    for j in acumulator_l:
        print("  ", j)
    print("  Mean:")
    new_left=np.mean(np.asarray(acumulator_l), axis=0)
    print(new_left)
        

print (acumulator_l)

[ -8.06974066e-05   9.60982310e-02   2.78752768e+02]
[  7.03926509e-06   1.94713299e-02   2.93125772e+02]
[  2.95774255e-04  -4.57536379e-01   5.00406200e+02]
[ -5.62877892e-04   8.53842219e-01   4.44515940e+01]
[  2.25840140e-04  -5.52171718e-01   6.02630228e+02]
[  2.83102524e-04  -3.89589233e-01   4.82462462e+02]
[ -2.30429280e-04  -1.03508845e-01   4.40129738e+02]
[ -5.55348452e-04   1.98300737e-01   4.54926083e+02]
8
Acumulator has 1 elements
   [ -8.06974066e-05   9.60982310e-02   2.78752768e+02]
  Mean:
[ -8.06974066e-05   9.60982310e-02   2.78752768e+02]
Acumulator has 2 elements
   [ -8.06974066e-05   9.60982310e-02   2.78752768e+02]
   [  7.03926509e-06   1.94713299e-02   2.93125772e+02]
  Mean:
[ -3.68290708e-05   5.77847805e-02   2.85939270e+02]
Acumulator has 3 elements
   [ -8.06974066e-05   9.60982310e-02   2.78752768e+02]
   [  7.03926509e-06   1.94713299e-02   2.93125772e+02]
   [  2.95774255e-04  -4.57536379e-01   5.00406200e+02]
  Mean:
[  7.40387046e-05  -1.13988939

In [9]:
dummy_l=history[1][0]
dummy_r=history[2][0]

dummy_l1=dummy_l*1.4
print(dummy_l)
print(dummy_l1)

diff=np.subtract(dummy_l, dummy_l1)
closedness=np.allclose(dummy_l[2], dummy_l1[2], atol=150)
print(diff)
print(closedness)

[  7.03926509e-06   1.94713299e-02   2.93125772e+02]
[  9.85497113e-06   2.72598619e-02   4.10376080e+02]
[ -2.81570604e-06  -7.78853197e-03  -1.17250309e+02]
True
